In [5]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import os
import math
from sklearn.model_selection import train_test_split,StratifiedKFold

## Feature Selection

### Import data

In [3]:
# define the function to perform feature engineering
def data_preprocessing(data_df):
    adata = ad.AnnData(X=data_df.values, 
                      obs=data_df.index.to_frame(), 
                      var=pd.DataFrame(index=data_df.columns))
    sc.pp.highly_variable_genes(adata, n_top_genes=5000, flavor='cell_ranger')
    adata_fselected = adata[:, adata.var['highly_variable']]
    return adata_fselected

In [6]:
# import datasets

## CellBench10x (5 lung cancer cell lines)
lc_data_df = pd.read_csv('../data/10x_5cl/10x_5cl_data.csv', index_col=0)
lc_label_df = pd.read_csv('../data/10x_5cl/Labels.csv',header=0) #the first row is header

In [7]:
## Baron(Human) (Human pancreas)
pan_data_df = pd.read_csv('../data/Baron Human/Filtered_Baron_HumanPancreas_data.csv', index_col=0)
pan_label_df = pd.read_csv('../data/Baron Human/Labels.csv',header=0)

In [8]:
## Zheng sorted	PBMC (immune system)
imm_data_df = pd.read_csv('../data/Zheng sorted/Filtered_DownSampled_SortedPBMC_data.csv', index_col=0)
imm_label_df = pd.read_csv('../data/Zheng sorted/Labels.csv',header=0)
imm_label_df

,x
0,CD14+ Monocyte
1,CD14+ Monocyte
2,CD14+ Monocyte
3,CD14+ Monocyte
4,CD14+ Monocyte
...,...
19995,CD8+/CD45RA+ Naive Cytotoxic
19996,CD8+/CD45RA+ Naive Cytotoxic
19997,CD8+/CD45RA+ Naive Cytotoxic
19998,CD8+/CD45RA+ Naive Cytotoxic


In [9]:
# feature selection
lc_adata_fselected = data_preprocessing(lc_data_df)
pan_adata_fselected = data_preprocessing(pan_data_df)
imm_adata_fselected = data_preprocessing(imm_data_df)

In [10]:
lc_adata_fselected

View of AnnData object with n_obs × n_vars = 3803 × 5000
    obs: 0
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'

# cross vaildation

#### After PCA

In [42]:
def create_cv_datasets(features, labels, output_root):
    """
    Create cross-validation datasets with an initial split into training and test datasets from CSV files,
    saving features and labels in separate CSV files.
    """

    # Split train and test (20%)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

    # Create Stratified K-Fold object
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    # Create directories and split data
    for fold_index, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train)):
        fold_dir = os.path.join(output_root, f'fold_{fold_index + 1}')
        os.makedirs(fold_dir, exist_ok=True)
        
        # Select train and validation data
        train_features = X_train.iloc[train_idx]
        valid_features = X_train.iloc[valid_idx]
        train_labels = y_train.iloc[train_idx].reset_index(drop=True)
        valid_labels = y_train.iloc[valid_idx].reset_index(drop=True)

        # Save to CSV
        train_features.to_csv(os.path.join(fold_dir, 'train_features.csv'), index=False)
        valid_features.to_csv(os.path.join(fold_dir, 'valid_features.csv'), index=False)
        train_labels.to_csv(os.path.join(fold_dir, 'train_labels.csv'), index=False)
        valid_labels.to_csv(os.path.join(fold_dir, 'valid_labels.csv'), index=False)

    # Save test set features and labels to separate CSV files
    X_test.to_csv(os.path.join(output_root, 'test_features.csv'), index=False)
    y_test.reset_index(drop=True).to_csv(os.path.join(output_root, 'test_labels.csv'), index=False)


In [27]:
output_dir = "../clean_data_pca_cv"

# Load data from CSV
lc_pca = pd.read_csv("../data_selected/lc_adata_fselected_31.csv",index_col=0)
imm_pca = pd.read_csv("../data_selected/imm_adata_fselected_107.csv",index_col=0)
pan_pca = pd.read_csv("../data_selected/pan_adata_fselected_8.csv",index_col=0)

In [44]:
create_cv_datasets(lc_pca,lc_label_df,output_dir+"/cellBench")

In [45]:
create_cv_datasets(pan_pca,pan_label_df,output_dir+"/baron")

/Applications/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  warnings.warn(


In [46]:
create_cv_datasets(imm_pca,imm_label_df,output_dir+"/zheng")

## No cv datasets
### no cv, just split train and test data, for active learning

In [25]:
def create_datasets(features, labels, output_root):

    # Split train and test (20%)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)
   
    os.makedirs(output_root, exist_ok=True)
    
    # Save test set features and labels to separate CSV files
    X_train.to_csv(os.path.join(output_root, 'train_features.csv'), index=False)
    y_train.reset_index(drop=True).to_csv(os.path.join(output_root, 'train_labels.csv'), index=False)
    X_test.to_csv(os.path.join(output_root, 'test_features.csv'), index=False)
    y_test.reset_index(drop=True).to_csv(os.path.join(output_root, 'test_labels.csv'), index=False)


In [26]:
output_dir = "../clean_data"

In [63]:
create_datasets(lc_pca,lc_label_df,output_dir+"/cellBench")

In [64]:
create_datasets(pan_pca,pan_label_df,output_dir+"/baron")

In [65]:
create_datasets(imm_pca,imm_label_df,output_dir+"/zheng")

## Kmeans

In [ ]:
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
# define the function to compute the Jaccard distances
# It will receive 2 arrays for clusters results
# and return the corresponding Jaccard distances between them
def compute_Jaccard(clusters1, clusters2):
    # Function to generate pairs using set
    def generate_pairs(clusters):
        pairs = set()
        for i in range(len(clusters) - 1):
            for j in range(i + 1, len(clusters)):
                if clusters[i] == clusters[j]:
                    pairs.add((i, j))  # Use tuple for immutable pair representation
        return pairs

    # Generate pairs for each clustering
    pairs1 = generate_pairs(clusters1)
    pairs2 = generate_pairs(clusters2)

    # Compute intersections and unions
    intersection = pairs1.intersection(pairs2)
    union = pairs1.union(pairs2)

    # Calculate Jaccard index
    if not union:  # handle the case when both pairs are empty
        return 1.0
    Jaccard_dis = len(intersection) / len(union)
    
    return Jaccard_dis

In [1]:
# Define the function to perform the Kmeans clustering
# It will return the objective values and the final clusters
# It will stop after converging
def perform_kmeans(mat, initial_center, k):
    center = initial_center.copy()
    obj = []
    cluster = np.zeros((mat.shape[0],1))
    check = True
    count = 0
    while check == True: # iteration loop
        
        obj_sum = 0
        
        for j in range(mat.shape[0]): # sample loop
            dis = []
            for c in range(k): # cluster loop
                d = math.sqrt(sum((mat[j,:]-center[c,:])**2))
                dis.append(d)
            min_idx = dis.index(min(dis))
            cluster[j, 0] = min_idx
        
        center = np.zeros((k, mat.shape[1]), float)
        for j in range(mat.shape[0]): # sample loop
            for c in range(k): # center loop
                if cluster[j, 0] == c:
                    center[c,:] += mat[j,:]
        
        for j in range(k): # center loop
            if np.count_nonzero(cluster == j) != 0:
                center[j, :] = center[j, :]/np.count_nonzero(cluster == j)
        
        for j in range(mat.shape[0]): # sample loop
            for c in range(k): # center loop
                if cluster[j,0] == c:
                    obj_sum += sum((mat[j,:]-center[c,:])**2)
        
        obj.append(obj_sum)
        
        # check converge
        if count != 0:
            if obj[count] == obj[count-1]:
                break
                
        count += 1
        
    return obj, cluster

In [3]:
# define the function to initialize the centroids
# it will return the centroids where each centroid's element is selected randomly 
# from the corresponding column of the original data 
def initialize_center(mat, k):
    center = np.zeros((k, mat.shape[1]))
    np.random.seed(620)
    for c in range(k):
        random_list = np.random.randint(0, mat.shape[0], size=mat.shape[0]).tolist()
        for j in range(mat.shape[1]):
            center[c,j] = mat[random_list[j],j]
    return center

### Data import

In [4]:
# get data after PCA
lc_pca = pd.read_csv('../data_selected/lc_adata_fselected_31.csv', index_col=0)
pan_pca = pd.read_csv('../data_selected/pan_adata_fselected_8.csv', index_col=0)
imm_pca = pd.read_csv('../data_selected/imm_adata_fselected_107.csv', index_col=0)

#### Perform Kmeans on three datastes after PCA

In [ ]:
def select_k(min_k, max_k, pca):
    obj_list = []
    for i in range(min_k, max_k+1):
        obj, kmeans_cluster = perform_kmeans(np.array(pca), initialize_center(np.array(pca), i), i)
        obj_list.append(obj[-1])
        print('Kmeans k = '+str(i)+' finished!')

    plt.plot([e for e in range(min_k, max_k+1)], obj_list, marker='o')
    plt.title('Objective Function Values at Convergence over different k', fontsize=12)
    plt.xlabel('k', fontsize=12)
    plt.ylabel('Objective function', fontsize=12)
    plt.xticks(range(min_k,max_k+1))
    plt.show()

In [7]:
# lc select k
select_k(1, 15, lc_pca)

In [ ]:
# lc kmeans
lc_obj, lc_kmeans_cluster = perform_kmeans(np.array(lc_pca), initialize_center(np.array(lc_pca), 5), 5)

In [ ]:
# lc get Jaccard index
codes, uniques = pd.factorize(lc_label_df.iloc[:,0])
lc_label_encode = np.array(pd.DataFrame({'Encoded': codes}))
lc_label_arr = np.array([item[0] for item in lc_label_encode])
lc_diff = compute_Jaccard(lc_label_arr, lc_kmeans_cluster)
print('The Jaccard similarity between clustering result and original label of lung cancer cells is', lc_diff)

In [8]:
# pan select k
select_k(1, 20, pan_pca)

In [ ]:
# pan kmeans
pan_obj, pan_kmeans_cluster = perform_kmeans(np.array(pan_pca), initialize_center(np.array(pan_pca), 14), 14)

In [ ]:
# pan get Jaccard index
codes, uniques = pd.factorize(pan_label_df.iloc[:,0])
pan_label_encode = np.array(pd.DataFrame({'Encoded': codes}))
pan_label_arr = np.array([item[0] for item in pan_label_encode])
pan_diff = compute_Jaccard(pan_label_arr, pan_kmeans_cluster)
print('The Jaccard similarity between clustering result and original label of pancreas cells is', pan_diff)

In [ ]:
# imm select k
select_k(1, 15, imm_pca)

In [ ]:
# imm kmeans
imm_obj, imm_kmeans_cluster = perform_kmeans(np.array(imm_pca), initialize_center(np.array(imm_pca), 10), 10)

In [ ]:
# imm get Jaccard index
codes, uniques = pd.factorize(imm_label_df.iloc[:,0])
imm_label_encode = np.array(pd.DataFrame({'Encoded': codes}))
imm_label_arr = np.array([item[0] for item in imm_label_encode])
imm_diff = compute_Jaccard(imm_label_arr, imm_kmeans_cluster)
print('The Jaccard similarity between clustering result and original label of immune system cells is', imm_diff)

### Visualization

In [ ]:
def get_label_name(label_num, label_df, label_arr):
    labels = []
    for i in range(label_num):
        label_name = label_df.iloc[np.where(label_arr == i)[0][0], 0]
        labels.append(label_name)
    return labels

In [ ]:
def reorder_cm(origin_cm):
    cost_matrix = -origin_cm
    
    # Solve the assignment problem
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # Reorder the confusion matrix
    reordered_cm = origin_cm[row_ind, :]
    reordered_cm = reordered_cm[:, col_ind]
    
    return reordered_cm

In [ ]:
def compute_f1_scores(cm):
    # True Positives are the diagonal elements
    TP = np.diag(cm)
    # False Positives are the sum of the column minus the diagonal
    FP = np.sum(cm, axis=0) - TP
    # False Negatives are the sum of the row minus the diagonal
    FN = np.sum(cm, axis=1) - TP
    
    # Precision for each class
    precision = np.divide(TP, TP + FP, out=np.zeros_like(TP, dtype=float), where=(TP+FP)!=0)
    # Recall for each class
    recall = np.divide(TP, TP + FN, out=np.zeros_like(TP, dtype=float), where=(TP+FN)!=0)
    
    # F1 Score for each class
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)  # Adding a small epsilon to avoid division by zero

    return f1_scores

In [ ]:
def compute_weighted_f1(cm):
    f1_scores = compute_f1_scores(cm)
    # Weighted-average F1 score
    weights = np.sum(cm, axis=1) / np.sum(cm)  # Class weights based on support
    weighted_f1 = np.average(f1_scores, weights=weights)
    
    return weighted_f1

In [ ]:
# lc kmeans visualization
lc_labels = get_label_name(5, lc_label_df, lc_label_arr)
lc_conf_matrix = confusion_matrix(lc_label_arr, lc_kmeans_cluster)
lc_reordered_cm = reorder_cm(lc_conf_matrix)

# plot the heatmap of reordered confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(lc_reordered_cm, annot=True, fmt="d", cmap="Blues", yticklabels=lc_labels)
plt.title("Confusion Matrix with Jaccard Index of Lung Cancer Cell Lines: {:.2f}".format(lc_diff), fontsize=14)
plt.xlabel("Predicted clusters", fontsize=12)
plt.ylabel("True labels", fontsize=12)
plt.show()

lc_f1 = compute_weighted_f1(lc_reordered_cm)
print('The weighted F1-score of Kmeans clustering on lung cancer cell lines is', lc_f1)

In [ ]:
# pan kmeans visualization
pan_labels = get_label_name(14, pan_label_df, pan_label_arr)
pan_conf_matrix = confusion_matrix(pan_label_arr, pan_kmeans_cluster)
pan_reordered_cm = reorder_cm(pan_conf_matrix)

# plot the heatmap of reordered confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(pan_reordered_cm, annot=True, fmt="d", cmap="Blues", yticklabels=pan_labels)
plt.title("Confusion Matrix with Jaccard Index of Pancreas Cells: {:.2f}".format(pan_diff), fontsize=14)
plt.xlabel("Predicted clusters", fontsize=12)
plt.ylabel("True labels", fontsize=12)
plt.show()

pan_f1 = compute_weighted_f1(pan_reordered_cm)
print('The weighted F1-score of Kmeans clustering on pancreas cells is', pan_f1)

In [ ]:
# imm kmeans visualization
imm_labels = get_label_name(10, imm_label_df, imm_label_arr)
imm_conf_matrix = confusion_matrix(imm_label_arr, imm_kmeans_cluster)
imm_reordered_cm = reorder_cm(imm_conf_matrix)

# plot the heatmap of reordered confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(imm_reordered_cm, annot=True, fmt="d", cmap="Blues", yticklabels=imm_labels)
plt.title("Confusion Matrix with Jaccard Index of Immune System Cells: {:.2f}".format(imm_diff), fontsize=14)
plt.xlabel("Predicted clusters", fontsize=12)
plt.ylabel("True labels", fontsize=12)
plt.show()

imm_f1 = compute_weighted_f1(imm_reordered_cm)
print('The weighted F1-score of Kmeans clustering on immune system cells is', imm_f1)

In [ ]:
# plot the Jaccard Index and F1-score of Kmeans

jacc_F1 = {
    'Dataset': ['Baron', 'Baron', 'CellBench', 'CellBench', 'Zheng', 'Zheng'],
    'Metric': ['Jaccard Index', 'F1-score', 'Jaccard Index', 'F1-score', 'Jaccard Index', 'F1-score'],
    'values': [0.23, 0.495124148777054, 0.34, 0.6346396368152479, 0.18, 0.3181956174307169]
}

jacc_F1_df = pd.DataFrame(jacc_F1)

# Pivot the data for plotting
df_pivot = jacc_F1_df.pivot(index='Dataset', columns='Metric', values='values')

# Plot
ax = df_pivot.plot(kind='bar', figsize=(8, 6), width=0.4)
plt.title('Evaluation for Kmeans on Three Datasets', fontsize=14)
plt.ylabel('Metric Values', fontsize=12)
plt.xlabel('Datasets', fontsize=12)
plt.xticks(rotation=0)
plt.legend(title='Metric types', bbox_to_anchor=(1, 1))
plt.tight_layout()

plt.show()

## LR

### To run the logistic regression model on cross validation dataset, JUST run these cmd:
#### bash run_lr_cv.sh baron
#### bash run_lr_cv.sh cellBench
#### bash run_lr_cv.sh zheng
### The bash script will automatically call the lr_final.py, then generate the metric.txt for each fold

In [10]:
# Calculate average metrics for the 10 fold

def calculate_average_metrics(directory):
    # Initilize DataFrame
    total_train_error = 0.0
    total_test_error = 0.0
    total_f1_score = 0.0
    file_count = 0

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                lines = file.readlines()
                if len(lines) == 3:
                    total_train_error += float(lines[0].split(',')[1])
                    total_test_error += float(lines[1].split(',')[1])
                    total_f1_score += float(lines[2].split(',')[1])
                    file_count += 1

    if file_count > 0:
        avg_train_error = total_train_error / file_count
        avg_test_error = total_test_error / file_count
        avg_f1_score = total_f1_score / file_count
    else:
        avg_train_error, avg_test_error, avg_f1_score = 0, 0, 0

    return avg_train_error, avg_test_error, avg_f1_score

In [21]:
directory = '../clean_data_pca/baron/metrics'

baron_averages = calculate_average_metrics(directory)

In [13]:
directory = '../clean_data_pca/cellBench/metrics'

cell_averages = calculate_average_metrics(directory)

In [14]:
directory = '../clean_data_pca/zheng/metrics'

zheng_averages = calculate_average_metrics(directory)

In [23]:
def write_csv(output,averages):
    with open(output, 'w') as file:
        file.write("Metric,Value\n")
        file.write(f"Train Error,{averages[0]}\n")
        file.write(f"Test Error,{averages[1]}\n")
        file.write(f"F1 Score,{averages[2]}\n")

In [24]:
write_csv("../LR_results/baron_avg_metrics.csv",baron_averages)
write_csv("../LR_results/cell_avg_metrics.csv",cell_averages)
write_csv("../LR_results/zheng_avg_metrics.csv",zheng_averages)